# Local PDF Questions
In this notebook we use `python 3.11.6` and `Ollama` to interact with PDFs from you local computer

## Table of Content
1. Setup
2. Ingesting PDFs
3. Embeddings
4. Retrieval


## Setup

In [1]:
!pip uninstall -y chromadb protobuf

Found existing installation: chromadb 1.0.9
Uninstalling chromadb-1.0.9:
  Successfully uninstalled chromadb-1.0.9
Found existing installation: protobuf 5.29.4
Uninstalling protobuf-5.29.4:
  Successfully uninstalled protobuf-5.29.4


In [2]:
!pip install tqdm "unstructured[all-docs]>=0.16.12" langchain chromadb 
!pip install langchain_community
!pip install langchain-text-splitters langchain-ollama protobuf

  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/6c/3b/9195db9e17516eb279a6adac3a4ca6062be5b922e4062b794e99dcccef91/chromadb-1.0.9-cp39-abi3-macosx_11_0_arm64.whl.metadata
  Using cached chromadb-1.0.9-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.9 kB)
  Obtaining dependency information for protobuf>=4.25.1 from https://files.pythonhosted.org/packages/21/46/33f884aa8bc59114dc97e0d954ca4618c556483670236008c88fbb7e834f/protobuf-6.31.0-cp39-abi3-macosx_10_9_universal2.whl.metadata
  Using cached protobuf-6.31.0-cp39-abi3-macosx_10_9_universal2.whl.metadata (593 bytes)
  Obtaining dependency information for protobuf>=4.25.1 from https://files.pythonhosted.org/packages/46/de/7c126bbb06aa0f8a7b38aaf8bd746c514d70e6a2a3f6dd460b3b7aad7aae/protobuf-5.29.4-cp38-abi3-macosx_10_9_universal2.whl.metadata
  Using cached protobuf-5.29.4-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
Using cached chromadb-1.0.9-cp39-abi3-macosx_11_0_arm64.whl (17.5 

## Ingesting PDFs
It is required to create on the same directory level a folder `pdfs` and in there store your desired `pdf` file to use. 
> Make sure to set the `pdf_name` variable below to the name of your pdf file

In [3]:
import os
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [4]:
pdf_name = "ReAct-AI-agents.pdf"
pdf_path = os.path.join("pdfs", pdf_name)
print(pdf_path)

if pdf_path:
    loader = UnstructuredPDFLoader(file_path=pdf_path)
    data = loader.load()
    print(f"Uploaded '{pdf_path}'.")
else:
    print("Upload a pdf in 'pdfs' folder. Create the folder if needed.")

pdfs/ReAct-AI-agents.pdf
Uploaded 'pdfs/ReAct-AI-agents.pdf'.


In [5]:
# Preview (output is too long, so only uncomment for curiosity)
# data[0].page_content

## Embeddings

This section requires that you have [Ollama](https://ollama.com/) installed in your local computer and that you have pulled an embedding model of your choice. For this notebook, we chose [nomic-embed-text](https://ollama.com/library/nomic-embed-text) as it has large context length and outperforms OpenAI's `text-embedding-ada-002` and `text-embedding-3-small`.

In [26]:
# pull the model in case you do not have it yet
!ollama serve
!ollama pull nomic-embed-text

Error: listen tcp 127.0.0.1:11434: bind: address already in use
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 970aa74c0a90: 100% ▕██████████████████▏ 274 MB                         
pulling c71d239df917: 100% ▕██████████████████▏  11 KB                         
pulling ce4a164fc046: 100% ▕██████████████████▏   17 B                         
pulling 31df23ea7daa: 100% ▕██████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [7]:
!ollama list

NAME                       ID              SIZE      MODIFIED               
nomic-embed-text:latest    0a109f422b47    274 MB    Less than a second ago    
mxbai-embed-large:335m     468836162de7    669 MB    2 hours ago               
nomic-embed-text:v1.5      0a109f422b47    274 MB    2 hours ago               
llama3.2:3b                a80c4f17acd5    2.0 GB    2 hours ago               


In [8]:
EMBEDDING_MODEL = "nomic-embed-text"

In [9]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [10]:
# Split and chunk pdf
text_splitter = RecursiveCharacterTextSplitter(chunk_size=8000, chunk_overlap=150)
chunks = text_splitter.split_documents(data)

In [11]:
# visualize two chunks (long output cell, uncomment to visualize if curious)
# chunks[:2]

[Document(metadata={'source': 'pdfs/ReAct-AI-agents.pdf'}, page_content='3 2 0 2\n\nr a\n\nM 0 1\n\n] L C . s c [\n\n3 v 9 2 6 3 0 . 0 1 2 2 : v i X r a\n\nPublished as a conference paper at ICLR 2023\n\nREACT: SYNERGIZING REASONING AND ACTING IN LANGUAGE MODELS\n\nShunyu Yao∗*,1, Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1, Yuan Cao2\n\n1Department of Computer Science, Princeton University 2Google Research, Brain team 1{shunyuy,karthikn}@princeton.edu 2{jeffreyzhao,dianyu,dunan,izhak,yuancao}@google.com\n\nABSTRACT\n\nWhile large language models (LLMs) have demonstrated impressive performance across tasks in language understanding and interactive decision making, their abilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g. action plan generation) have primarily been studied as separate topics. In this paper, we explore the use of LLMs to generate both reasoning traces and task-speciﬁc actions in an interleaved manner, allowing for greater 

In [12]:
# Add chunks to vector database (chunks get embeded)
vdb = Chroma.from_documents(
    documents=chunks,
    persist_directory=None, # to Chroma an epehemral client aka in memory
    embedding=OllamaEmbeddings(model=EMBEDDING_MODEL),
    collection_name="local_rag", # collection created in chromadb
)

## Retrieval 

In [13]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [14]:
# LLM from Ollama
LOCAL_MODEL = "llama3.2:3b"

In [15]:
llm = ChatOllama(model=LOCAL_MODEL)

In [16]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate 2
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [17]:
# Retriever
retriever = MultiQueryRetriever.from_llm(
    vdb.as_retriever(),
    llm,
    prompt=QUERY_PROMPT,
)

In [18]:
# RAG prompt
template = """
Answer the question based ONLY on the following context:
{context}
Question: 
{question}
"""

rag_prompt = ChatPromptTemplate.from_template(template)

In [19]:
# create a chain to run all pipeline of steps
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [21]:
# call the whole pipeline of steps aka do RAG
# uncomment this for getting an input to type your question
response = chain.invoke(input(""))

 What is this doc about?


In [23]:
response

"This document appears to be a conference paper or research publication in the field of artificial intelligence (AI) and natural language processing (NLP). Specifically, it discusses a novel prompt-based paradigm called ReAct that aims to synergize reasoning and acting in language models for general task solving.\n\nThe paper presents the concept of ReAct, which augments an agent's action space by incorporating a language space, allowing it to generate both domain-specific actions and free-form language thoughts. The authors demonstrate the effectiveness of ReAct on various knowledge-intensive reasoning tasks, including multi-hop question answering and fact verification.\n\nThe document also discusses the benefits of ReAct, including its intuitive design, flexibility, performance, robustness, human alignment, and controllability. Additionally, it provides experimental results and analysis to support the claims made in the paper.\n\nOverall, this document appears to be a research paper 

In [24]:
resp2 = chain.invoke("What are the main 5 take aways from the document?")
resp2

'Based on the document, here are five main takeaways:\n\n1. **ReAct is a novel prompt-based paradigm that synergizes reasoning and acting in large language models**: ReAct introduces a simple yet effective method for combining reasoning and action generation to improve performance in general task solving.\n\n2. **ReAct shows strong generalization and robustness across diverse tasks and domains**: The approach consistently outperforms baselines with only reasoning or action generation, and demonstrates robustness to prompt selections, limited support of reasoning and acting behaviors, and high-quality human annotations.\n\n3. **ReAct enjoys several unique features that make it easy to design, generalizable, performant, and human-aligned**: Designing ReAct prompts is straightforward, the approach works for diverse tasks with distinct action spaces and reasoning needs, and humans can easily inspect reasoning traces and control or correct agent behavior on the go.\n\n4. **ReAct demonstrate

In [25]:
# Delete all collections in the vector db 
# (aka clean memory because chroma was ephemeral)
vdb.delete_collection()